[![GitHub](https://img.shields.io/badge/Github-hibana2077-blue?style=plastic-square&logo=github)](https://github.com/hibana2077)
[![Colab](https://img.shields.io/badge/Colab-Open%20in%20Colab-blue?style=plastic-square&logo=googlecolab)](https://colab.research.google.com/github/hibana2077/hibana2077/blob/master/train/train.ipynb)

如果要訓練這份資料集會需要安裝talib套件，請參考[這裡](https://www.lfd.uci.edu/~gohlke/pythonlibs/#ta-lib)下載對應的版本，並使用pip安裝。

In [ ]:
# !pip install ccxt

In [ ]:
# !wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
# !tar -xzvf ta-lib-0.4.0-src.tar.gz
# %cd ta-lib
# !./configure --prefix=/usr
# !make
# !make install
# !pip install Ta-Lib

In [1]:
from ccxt import binance
from torch.utils.data import Dataset, DataLoader
from torch import nn
from torch import optim
from torch.nn import functional as F
from talib import abstract
import pandas as pd
import numpy as np
import torch
import os
import sys

# 環境整理

In [2]:
ls_dir = os.listdir(path="..")
if "data" not in ls_dir:
    os.mkdir(path="../data")
if "model" not in ls_dir:
    os.mkdir(path="../model")
if "data" in ls_dir:
    ls_dir = os.listdir(path="../data")
    #remove all files in data folder
    for file in ls_dir:os.remove(path="../data/"+file)

# 下載資料集

In [3]:
# Binance BTC/USDT 1h candles from 2020-01-01 to 2021-01-01

binance = binance()
symbol = 'ADA/USDT'
timeframe = '15m'
file_name = f"../data/{symbol.replace('/', '_')}_{timeframe}.csv"
start = binance.parse8601('2022-10-01T00:00:00Z')
end = binance.parse8601('2022-12-30T00:00:00Z')
cnt_time = start
data = []
while cnt_time < end:
    ohlcv = binance.fetch_ohlcv(symbol, timeframe, cnt_time)
    data += ohlcv
    cnt_time = ohlcv[-1][0] + 3600000 # 1h in ms    
df = pd.DataFrame(data, columns=['time', 'open', 'high', 'low', 'close', 'volume'])
df['time'] = pd.to_datetime(df['time'], unit='ms')
df.to_csv(file_name, index=False)

# 讀取資料集

In [ ]:
#如果有下载好的數據，可以直接讀取
data_file = '../data/btc_usdt_1h.csv' #-> 可以自行更換
df = pd.read_csv(data_file)

# 數據處理

- 計算RSI
- 計算MACD
- 計算OBV
- 計算CCI
- 改成變化百分比 -> 標準化

關於技術指標的說明可以參考[這裡](https://www.investopedia.com/terms/t/technicalindicator.asp)，或是google。

In [4]:
df['RSI'] = abstract.RSI(df, timeperiod=14)
df['MACD'] = abstract.MACD(df, fastperiod=12, slowperiod=26, signalperiod=9)['macd'] #只取MACD
df['OBV'] = abstract.OBV(df, timeperiod=14)
df['CCI'] = abstract.CCI(df, timeperiod=14)
df['ATR'] = abstract.ATR(df, timeperiod=14)
df['ADX'] = abstract.ADX(df, timeperiod=14)
df['MFI'] = abstract.MFI(df, timeperiod=14)
df['CLOSE_percent'] = df['close'].pct_change()
#由於RSI MACD OBV CCI 他們已經是標準化的，所以不需要再標準化

# 設定買賣點

將買賣點分為下跌、上漲、不動，並將數據轉成one-hot編碼。

In [5]:
df['UP'] = df['CLOSE_percent'].apply(lambda x: 1 if x > 0 else 0)
df['DOWN'] = df['CLOSE_percent'].apply(lambda x: 1 if x < 0 else 0)
df['UP'] = df['UP'].shift(-1) #shift UP DOWN 一個單位，因為我們要預測的是下一個時間點的漲跌
df['DOWN'] = df['DOWN'].shift(-1)

df = df.dropna()


In [6]:
df

,time,open,high,low,close,volume,RSI,MACD,OBV,CCI,ATR,ADX,MFI,CLOSE_percent,UP,DOWN
33,2019-10-01 08:15:00,8397.80,8399.84,8355.00,8356.53,383.065769,49.371020,22.323754,3364.579414,-70.616834,39.812174,29.333975,23.902832,-0.004912,1.0,0.0
34,2019-10-01 08:30:00,8357.33,8375.29,8345.00,8366.91,476.302794,51.246753,18.889651,3840.882208,-86.262910,39.132018,27.373650,17.775859,0.001242,1.0,0.0
35,2019-10-01 08:45:00,8366.57,8381.38,8363.79,8376.28,235.717562,52.941620,16.731313,4076.599770,-40.451685,37.593303,25.482399,23.351751,0.001120,0.0,1.0
36,2019-10-01 09:00:00,8377.68,8378.41,8290.00,8319.34,710.028749,43.129386,10.307414,3366.571021,-191.047624,41.223067,25.493180,21.364349,-0.006798,0.0,1.0
37,2019-10-01 09:15:00,8320.00,8330.04,8290.00,8305.68,613.360759,41.158558,4.067295,2753.210262,-226.560909,41.138562,25.503190,15.454057,-0.001642,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113243,2023-01-01 09:30:00,16514.64,16533.12,16514.22,16531.85,883.551270,52.745660,-4.536439,-56979.621176,9.296856,15.283688,31.376614,32.260504,0.001067,1.0,0.0
113244,2023-01-01 09:45:00,16531.85,16542.49,16525.62,16537.88,1213.731820,55.918864,-2.963106,-55765.889356,86.704775,15.396996,29.232544,33.333444,0.000365,1.0,0.0
113245,2023-01-01 10:00:00,16538.32,16543.14,16533.42,16542.85,1179.055140,58.398506,-1.300204,-54586.834216,128.445049,14.991496,27.181290,39.936794,0.000301,1.0,0.0
113246,2023-01-01 10:15:00,16542.60,16547.38,16539.98,16544.07,801.487390,59.008075,0.114779,-53785.346826,157.054176,14.449247,25.600024,44.888346,0.000074,0.0,1.0


In [7]:
#正規化
df['RSI'] = (df['RSI'] - df['RSI'].min()) / (df['RSI'].max() - df['RSI'].min())
df['MACD'] = (df['MACD'] - df['MACD'].min()) / (df['MACD'].max() - df['MACD'].min())
df['OBV'] = (df['OBV'] - df['OBV'].min()) / (df['OBV'].max() - df['OBV'].min())
df['CCI'] = (df['CCI'] - df['CCI'].min()) / (df['CCI'].max() - df['CCI'].min())
df['ATR'] = (df['ATR'] - df['ATR'].min()) / (df['ATR'].max() - df['ATR'].min())
df['ADX'] = (df['ADX'] - df['ADX'].min()) / (df['ADX'].max() - df['ADX'].min())
df['MFI'] = (df['MFI'] - df['MFI'].min()) / (df['MFI'].max() - df['MFI'].min())
df['open'] = (df['open'] - df['open'].min()) / (df['open'].max() - df['open'].min())
df['high'] = (df['high'] - df['high'].min()) / (df['high'].max() - df['high'].min())
df['low'] = (df['low'] - df['low'].min()) / (df['low'].max() - df['low'].min())
df['close'] = (df['close'] - df['close'].min()) / (df['close'].max() - df['close'].min())
df['volume'] = (df['volume'] - df['volume'].min()) / (df['volume'].max() - df['volume'].min())

C:\Users\李軒豪\AppData\Local\Temp\ipykernel_23024\1144569331.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['RSI'] = (df['RSI'] - df['RSI'].min()) / (df['RSI'].max() - df['RSI'].min())
C:\Users\李軒豪\AppData\Local\Temp\ipykernel_23024\1144569331.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['MACD'] = (df['MACD'] - df['MACD'].min()) / (df['MACD'].max() - df['MACD'].min())
C:\Users\李軒豪\AppData\Local\Temp\ipykernel_23024\1144569331.py:4: SettingWithCopyWarning: 
A value is trying to be set on a co

In [8]:
df.drop(['CLOSE_percent'], axis=1, inplace=True)

C:\Users\李軒豪\AppData\Roaming\Python\Python310\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [9]:
df

,time,open,high,low,close,volume,RSI,MACD,OBV,CCI,ATR,ADX,MFI,UP,DOWN
33,2019-10-01 08:15:00,0.069553,0.064224,0.070680,0.069009,0.009489,0.494393,0.589019,0.356367,0.424339,0.017151,0.332065,0.239028,1.0,0.0
34,2019-10-01 08:30:00,0.068929,0.063845,0.070526,0.069169,0.011798,0.514935,0.587832,0.357083,0.407575,0.016798,0.304525,0.177759,1.0,0.0
35,2019-10-01 08:45:00,0.069071,0.063939,0.070816,0.069314,0.005839,0.533497,0.587086,0.357438,0.456659,0.016002,0.277955,0.233518,0.0,1.0
36,2019-10-01 09:00:00,0.069243,0.063893,0.069676,0.068435,0.017587,0.426038,0.584865,0.356370,0.295306,0.017881,0.278106,0.213643,0.0,1.0
37,2019-10-01 09:15:00,0.068353,0.063146,0.069676,0.068225,0.015193,0.404455,0.582708,0.355448,0.257256,0.017838,0.278247,0.154541,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113243,2023-01-01 09:30:00,0.194755,0.189816,0.196793,0.195100,0.021886,0.531351,0.579733,0.265645,0.509961,0.004448,0.360762,0.322605,1.0,0.0
113244,2023-01-01 09:45:00,0.195020,0.189961,0.196969,0.195193,0.030064,0.566102,0.580277,0.267470,0.592898,0.004507,0.330640,0.333334,1.0,0.0
113245,2023-01-01 10:00:00,0.195120,0.189971,0.197090,0.195270,0.029205,0.593258,0.580852,0.269243,0.637620,0.004297,0.301822,0.399368,1.0,0.0
113246,2023-01-01 10:15:00,0.195186,0.190037,0.197191,0.195288,0.019853,0.599933,0.581341,0.270448,0.668272,0.004016,0.279608,0.448883,0.0,1.0


In [10]:
df['UP'].value_counts()

1.0    56969
0.0    56246
Name: UP, dtype: int64

In [11]:
df['DOWN'].value_counts()

0.0    57001
1.0    56214
Name: DOWN, dtype: int64

看起來數據蠻平衡的

# 儲存資料

In [12]:
df.to_csv(file_name, index=False)

# 分割成X、y

In [13]:

X,y = list(),list()
ref_bar = 10

for i in range(len(df)-ref_bar):
    #df.iloc.values 會回傳一個numpy array
    X.append(df.iloc[i:i+ref_bar, 1:13].values) # i to i+ref_bar-1
    y.append(df.iloc[i+ref_bar-1, 13:].values) # i+ref_bar-1



In [14]:
X[0]#這裡會出現10個array，每個array裡面有12個數字，分別是open high low close volume RSI MACD OBV CCI ATR ADX MFI

array([[0.06955319, 0.06422388, 0.07068038, 0.06900893, 0.00948854,
        0.49439341, 0.58901944, 0.35636723, 0.42433911, 0.0171507 ,
        0.33206521, 0.23902832],
       [0.06892894, 0.06384478, 0.07052581, 0.06916903, 0.01179802,
        0.51493541, 0.58783218, 0.35708331, 0.40757545, 0.01679846,
        0.30452492, 0.17775859],
       [0.06907147, 0.06393882, 0.07081624, 0.06931354, 0.00583873,
        0.53349668, 0.58708598, 0.35743769, 0.45665891, 0.01600159,
        0.27795504, 0.23351751],
       [0.06924284, 0.06389296, 0.06967571, 0.06843534, 0.01758742,
        0.42603842, 0.58486506, 0.35637022, 0.29530612, 0.01788137,
        0.27810649, 0.21364349],
       [0.06835313, 0.06314604, 0.06967571, 0.06822465, 0.01519295,
        0.40445498, 0.58270768, 0.35544809, 0.25725617, 0.01783761,
        0.27824712, 0.15454057],
       [0.06813225, 0.06330802, 0.06976876, 0.06848084, 0.01153835,
        0.44083766, 0.5814595 , 0.35614841, 0.34607659, 0.01796232,
        0.27313944,

In [15]:
y[0]

array([0.0, 1.0], dtype=object)

In [16]:
X = np.array(X)
X = torch.tensor(X, dtype=torch.float32)

In [17]:
y = np.array(y,dtype=np.float32)
y = torch.tensor(y, dtype=torch.float32)

In [18]:
y

tensor([[0., 1.],
        [1., 0.],
        [1., 0.],
        ...,
        [1., 0.],
        [1., 0.],
        [0., 1.]])

In [19]:
print(f"X shape: {X.shape} , y shape: {y.shape}")

X shape: torch.Size([113205, 10, 12]) , y shape: torch.Size([113205, 2])


In [20]:
X[0]

tensor([[0.0696, 0.0642, 0.0707, 0.0690, 0.0095, 0.4944, 0.5890, 0.3564, 0.4243,
         0.0172, 0.3321, 0.2390],
        [0.0689, 0.0638, 0.0705, 0.0692, 0.0118, 0.5149, 0.5878, 0.3571, 0.4076,
         0.0168, 0.3045, 0.1778],
        [0.0691, 0.0639, 0.0708, 0.0693, 0.0058, 0.5335, 0.5871, 0.3574, 0.4567,
         0.0160, 0.2780, 0.2335],
        [0.0692, 0.0639, 0.0697, 0.0684, 0.0176, 0.4260, 0.5849, 0.3564, 0.2953,
         0.0179, 0.2781, 0.2136],
        [0.0684, 0.0631, 0.0697, 0.0682, 0.0152, 0.4045, 0.5827, 0.3554, 0.2573,
         0.0178, 0.2782, 0.1545],
        [0.0681, 0.0633, 0.0698, 0.0685, 0.0115, 0.4408, 0.5815, 0.3561, 0.3461,
         0.0180, 0.2731, 0.2408],
        [0.0683, 0.0631, 0.0699, 0.0684, 0.0070, 0.4318, 0.5803, 0.3557, 0.3708,
         0.0174, 0.2684, 0.2446],
        [0.0683, 0.0635, 0.0701, 0.0688, 0.0086, 0.4851, 0.5801, 0.3562, 0.4450,
         0.0173, 0.2514, 0.3154],
        [0.0687, 0.0636, 0.0702, 0.0686, 0.0084, 0.4608, 0.5797, 0.3557, 0.4541,

In [21]:
y[0]

tensor([0., 1.])

# 建立資料集類別

- 要繼承torch.utils.data.Dataset
- 要實作`__len__`、`__getitem__`
- 後面要用DataLoader取用

In [22]:
#用sklearn來分成train val test
# train:val:test = 6:3:1
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.75, random_state=42)

In [23]:
print(f"X_train shape: {X_train.shape} , y_train shape: {y_train.shape}")

X_train shape: torch.Size([67923, 10, 12]) , y_train shape: torch.Size([67923, 2])


In [24]:
print(f"X_test shape: {X_test.shape} , y_test shape: {y_test.shape}")

X_test shape: torch.Size([33962, 10, 12]) , y_test shape: torch.Size([33962, 2])


In [25]:
print(f"X_val shape: {X_val.shape} , y_val shape: {y_val.shape}")

X_val shape: torch.Size([11320, 10, 12]) , y_val shape: torch.Size([11320, 2])


In [26]:
class TrainDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
    def __len__(self):
        return len(self.X)
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [27]:
class ValDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
    def __len__(self):
        return len(self.X)
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [28]:
class TestDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
    def __len__(self):
        return len(self.X)
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [29]:
train_dataset = TrainDataset(X_train, y_train)
val_dataset = ValDataset(X_val, y_val)
test_dataset = TestDataset(X_test, y_test)

# 建立模型

- 要繼承torch.nn.Module
- 可能要多建立不同的模型，到時候看結果再調整 -> 先讓數據流得通，再去看成績做調整。

- ver1 -> CNN+MLP
- ver2 -> CNN+LSTM+MLP
- ver3 -> CNN+GRU+MLP
- ver4 -> CNN+LSTM+GRU+MLP
- ver5 -> CNN+GRU+MLP+Dropout

In [30]:
class SelectItem(torch.nn.Module):#這是用來取出多個輸出其中一個的輸出，如果不用sequential的話，就可以不用這個
    def __init__(self, item_index):
        super(SelectItem, self).__init__()
        self._name = 'selectitem'
        self.item_index = item_index

    def forward(self, inputs):
        return inputs[self.item_index]

In [31]:
class crypto_classfier_ver1(nn.Module): #CNN+MLP
    def __init__(self):
        super(crypto_classfier_ver1, self).__init__()
        self.name = "CCV-1"
        self.spare_layer = nn.Sequential(
            torch.nn.Conv1d(10, 4, 3, stride=4, padding=1),
            torch.nn.ReLU())
        self.net = nn.Sequential(
            torch.nn.Linear(4, 16),
            torch.nn.ReLU(),
            torch.nn.Linear(16, 128),
            torch.nn.ReLU(),
            torch.nn.Linear(128, 512),
            torch.nn.Dropout(0.2),
            torch.nn.ReLU(),
            torch.nn.Linear(512, 1024), 
            torch.nn.Dropout(0.2),
            torch.nn.ReLU(),
            torch.nn.Linear(1024, 512),
            torch.nn.Dropout(0.2),
            torch.nn.ReLU(),
            torch.nn.Linear(512, 128),
            torch.nn.ReLU(),
            torch.nn.Linear(128, 16),
            torch.nn.ReLU(),
            torch.nn.Linear(16, 2),
            torch.nn.ReLU())
        self.fusion = nn.Sequential(
            torch.nn.Linear(6, 16),
            torch.nn.ELU(),
            torch.nn.Linear(16, 2),
            torch.nn.Softmax(dim=0))

    def forward(self, x):
        x = self.spare_layer(x)
        x = x.T
        x1 = self.net(x[0])
        x2 = self.net(x[1])
        x3 = self.net(x[2])
        x = torch.cat((x1,x2,x3),0)
        x = self.fusion(x)
        return x

In [32]:
class crypto_classfier_ver2(nn.Module): #CNN+LSTM+MLP
    def __init__(self):
        super(crypto_classfier_ver2, self).__init__()
        self.name = "CCV-2"
        self.net = nn.Sequential(
            torch.nn.Conv1d(10, 20, 3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool1d(1, stride=1),
            torch.nn.Conv1d(20, 40, 3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool1d(1, stride=1),
            torch.nn.Conv1d(40, 1, 3, stride=1, padding=1),
            torch.nn.Linear(12,64),
            torch.nn.Linear(64,128),
            torch.nn.LSTM(128, 64, 25),
            SelectItem(0),
            SelectItem(0),
            torch.nn.Linear(64,2),
            torch.nn.Softmax(dim=0))
    def forward(self, x):
        x = self.net(x)
        return x

In [33]:
class crypto_classfier_ver3(nn.Module):#CNN+GRU+MLP
    def __init__(self):
        super(crypto_classfier_ver3, self).__init__()
        self.name = "CCV-3"
        self.net = nn.Sequential(
            torch.nn.Conv1d(10, 20, 3, stride=1, padding=1),
            torch.nn.ELU(),
            torch.nn.MaxPool1d(1, stride=1),
            torch.nn.Conv1d(20, 40, 3, stride=1, padding=1),
            torch.nn.ELU(),
            torch.nn.MaxPool1d(1, stride=1),
            torch.nn.Conv1d(40, 1, 3, stride=1, padding=1),
            torch.nn.Linear(12,64),
            torch.nn.Linear(64,128),
            torch.nn.GRU(128, 64, 25),
            SelectItem(0),
            SelectItem(0),
            torch.nn.Linear(64,2),
            torch.nn.ELU()
            )
    def forward(self, x):
        x = self.net(x)
        # print(f"=>{F.softmax(x, dim=0)}")
        x = F.softmax(x, dim=0)
        return x

In [34]:
class crypto_classfier_ver4(nn.Module):#CNN+LSTM+GRU+MLP
    def __init__(self):
        super(crypto_classfier_ver4, self).__init__()
        self.name = "CCV-4"
        self.batch_size = 1
        self.spare_layer = torch.nn.Conv1d(10, 2, 3, stride=3, padding=1)
        self.gru = nn.Sequential(
            torch.nn.Linear(4, 16),
            torch.nn.ELU(),
            torch.nn.Linear(16, 128),
            torch.nn.ELU(),
            torch.nn.GRU(128, 64, 25),
            SelectItem(0),
            torch.nn.Linear(64, 2))
        self.lstm = nn.Sequential(
            torch.nn.Linear(4, 16),
            torch.nn.ELU(),
            torch.nn.Linear(16, 128),
            torch.nn.ELU(),
            torch.nn.LSTM(128, 256, 25),
            SelectItem(0),
            torch.nn.Dropout(0.2),
            torch.nn.LSTM(256, 512, 25),
            SelectItem(0),
            torch.nn.Dropout(0.2),
            torch.nn.LSTM(512, 256, 25),
            SelectItem(0),
            torch.nn.Dropout(0.2),
            torch.nn.LSTM(256, 128, 25),
            SelectItem(0),
            torch.nn.Dropout(0.2),
            torch.nn.LSTM(128, 64, 25),
            SelectItem(0),
            torch.nn.Linear(64, 2))
        self.fusion = nn.Sequential(
            SelectItem(0),
            SelectItem(0),
            torch.nn.Linear(4, 16),
            torch.nn.ELU(),
            torch.nn.Linear(16, 2),
            torch.nn.Softmax(dim=0)
            )
    def forward(self, x):
        x = self.spare_layer(x)
        x1 , x2 = x[0], x[1]
        x1 = x1.view(1,1,4)
        x2 = x2.view(1,1,4)
        out1 = self.gru(x1)
        out2 = self.lstm(x2)
        x = torch.cat((out1,out2),2)
        x = self.fusion(x)
        return x


In [35]:
class CNNwithDropout(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(CNNwithDropout, self).__init__()
        self.net = nn.Sequential(
            nn.Conv1d(in_channels, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(2),
            nn.Conv1d(16, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(2),
            nn.Conv1d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(2),
            nn.Flatten(),
            nn.Linear(1 , 128),
            nn.Linear(128, out_channels)
        )
        self.output_layer = nn.Sequential(
            nn.Conv1d(64, 1, kernel_size=3, padding=1),
            SelectItem(0),
            nn.Linear(2,16),
            nn.Linear(16,64),
            nn.ELU(),
            nn.Dropout(0.5),
            nn.Linear(64,128),
            nn.Linear(128,2),
            nn.ELU(),
            nn.Softmax(dim=0)
        )

    def forward(self, x):
        return self.output_layer(self.net(x))

In [36]:
class crypto_classfier_ver5(nn.Module):#CNN+GRU+MLP
    def __init__(self):
        super(crypto_classfier_ver5, self).__init__()
        self.name = "CCV-5"
        self.net = nn.Sequential(
            torch.nn.Conv1d(10, 20, 3, stride=1, padding=1),
            torch.nn.ELU(),
            torch.nn.MaxPool1d(1, stride=1),
            torch.nn.Conv1d(20, 40, 3, stride=1, padding=1),
            torch.nn.ELU(),
            torch.nn.MaxPool1d(1, stride=1),
            torch.nn.Conv1d(40, 1, 3, stride=1, padding=1),
            torch.nn.Linear(12,64),
            torch.nn.Linear(64,128),
            torch.nn.GRU(128, 64, 25),
            SelectItem(0),
            torch.nn.Dropout(0.5),
            torch.nn.GRU(64, 32, 25),
            SelectItem(0),
            torch.nn.Dropout(0.5),
            torch.nn.GRU(32, 16, 25),
            SelectItem(0),
            SelectItem(0),
            torch.nn.Dropout(0.5),
            torch.nn.Linear(16,2),
            torch.nn.Softmax(dim=0)
            )
    def forward(self, x):
        x = self.net(x)
        return x

In [37]:
lab_tensor = train_dataset[0][0]
lab_ans_tensor = train_dataset[0][1]
print(lab_tensor.shape)
print(lab_ans_tensor.shape)
print(lab_tensor)
print(lab_ans_tensor)

torch.Size([10, 12])
torch.Size([2])
tensor([[0.0982, 0.0933, 0.1000, 0.0987, 0.0118, 0.5241, 0.5753, 0.4052, 0.6211,
         0.0128, 0.3322, 0.4336],
        [0.0987, 0.0934, 0.1001, 0.0985, 0.0111, 0.4818, 0.5760, 0.4045, 0.6294,
         0.0130, 0.3030, 0.4262],
        [0.0984, 0.0931, 0.0999, 0.0983, 0.0097, 0.4599, 0.5764, 0.4039, 0.5574,
         0.0126, 0.2822, 0.4428],
        [0.0982, 0.0932, 0.1000, 0.0984, 0.0062, 0.4655, 0.5768, 0.4043, 0.5823,
         0.0123, 0.2579, 0.5099],
        [0.0983, 0.0933, 0.1001, 0.0984, 0.0101, 0.4657, 0.5772, 0.4049, 0.6017,
         0.0122, 0.2399, 0.5134],
        [0.0983, 0.0934, 0.1001, 0.0987, 0.0078, 0.5234, 0.5781, 0.4054, 0.6201,
         0.0121, 0.2246, 0.5944],
        [0.0986, 0.0933, 0.1002, 0.0986, 0.0089, 0.5176, 0.5787, 0.4048, 0.6047,
         0.0116, 0.2104, 0.7049],
        [0.0986, 0.0932, 0.0998, 0.0981, 0.0109, 0.4183, 0.5783, 0.4042, 0.4872,
         0.0120, 0.2023, 0.6055],
        [0.0980, 0.0931, 0.0998, 0.0984, 0.

In [38]:
lab_models = crypto_classfier_ver4()
lab_out = lab_models(lab_tensor)
print(lab_out)

tensor([0.5813, 0.4187], grad_fn=<SoftmaxBackward0>)


In [39]:
lab_target = torch.tensor([0.,1.])
lab_test_ansers = [torch.tensor([1-(i*0.1),i*0.1]) for i in range(1,10)]

In [40]:
for i in lab_test_ansers:
    print(f"BCE loss:{F.binary_cross_entropy(i, lab_target):.4f} ans:{i}")
    print(f"CE loss:{F.cross_entropy(i.view(1,2), torch.tensor([1])):.4f} ans:{i}")
    print(f"MSE loss:{F.mse_loss(i, lab_target):.4f} ans:{i}")
    print("=====================================")

BCE loss:2.3026 ans:tensor([0.9000, 0.1000])
CE loss:1.1711 ans:tensor([0.9000, 0.1000])
MSE loss:0.8100 ans:tensor([0.9000, 0.1000])
BCE loss:1.6094 ans:tensor([0.8000, 0.2000])
CE loss:1.0375 ans:tensor([0.8000, 0.2000])
MSE loss:0.6400 ans:tensor([0.8000, 0.2000])
BCE loss:1.2040 ans:tensor([0.7000, 0.3000])
CE loss:0.9130 ans:tensor([0.7000, 0.3000])
MSE loss:0.4900 ans:tensor([0.7000, 0.3000])
BCE loss:0.9163 ans:tensor([0.6000, 0.4000])
CE loss:0.7981 ans:tensor([0.6000, 0.4000])
MSE loss:0.3600 ans:tensor([0.6000, 0.4000])
BCE loss:0.6931 ans:tensor([0.5000, 0.5000])
CE loss:0.6931 ans:tensor([0.5000, 0.5000])
MSE loss:0.2500 ans:tensor([0.5000, 0.5000])
BCE loss:0.5108 ans:tensor([0.4000, 0.6000])
CE loss:0.5981 ans:tensor([0.4000, 0.6000])
MSE loss:0.1600 ans:tensor([0.4000, 0.6000])
BCE loss:0.3567 ans:tensor([0.3000, 0.7000])
CE loss:0.5130 ans:tensor([0.3000, 0.7000])
MSE loss:0.0900 ans:tensor([0.3000, 0.7000])
BCE loss:0.2231 ans:tensor([0.2000, 0.8000])
CE loss:0.4375 an

In [41]:
lab_models = [crypto_classfier_ver1(), crypto_classfier_ver2(), crypto_classfier_ver3(), crypto_classfier_ver4(), crypto_classfier_ver5(), CNNwithDropout(10,2)]
out_data = list(map(lambda x: x(lab_tensor), lab_models))

for t in out_data:
    print(t)

tensor([0.5549, 0.4451], grad_fn=<SoftmaxBackward0>)
tensor([0.5041, 0.4959], grad_fn=<SoftmaxBackward0>)
tensor([0.4960, 0.5040], grad_fn=<SoftmaxBackward0>)
tensor([0.3447, 0.6553], grad_fn=<SoftmaxBackward0>)
tensor([0.4212, 0.5788], grad_fn=<SoftmaxBackward0>)
tensor([0.4900, 0.5100], grad_fn=<SoftmaxBackward0>)


## Note

1) model
    - `__init__`: define [layers](https://pytorch.org/docs/stable/nn.html)
    - forward: forward pass -> compute prediction
2) loss and optimizer
    - lr: learning rate [default=0.001]
    - momentum: momentum for optimizer [default=0.9]
    - criterion: loss function [in torch.nn] eg.nn.BCELoss()
    - optimizer: optimizer [in torch.optim] eg.torch.optim.SGD()
        - eg. optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum)
3) training loop
    - forward pass: compute prediction and loss

        ```python
        output = model(data)
        loss = criterion(output, target)
        ```
        
    - backward pass: loss.backward()
    - update weights: optimizer.step()
    - zero the gradients: optimizer.zero_grad()

# TODO

- 寫訓練方法
    - using dataloader
        - batch and epoch
- 寫驗證方法
    - using model.eval()


In [43]:
test_dataloader_temp = DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=0)
loss_f = nn.MSELoss()
test_model = crypto_classfier_ver1()
for ind,(x,y) in enumerate(test_dataloader_temp):
    if ind == 50:
        print(x.shape)
        print(y.shape)
        x = x[0]
        y = y[0]
        out= test_model(x)
        out = torch.tensor([1.,0.]) if out[0] > out[1] else torch.tensor([0.,1.])
        loss_val = loss_f(out, y)
        print(loss_val)
        print(out.shape)
        print(out)
        print(y)
        break

torch.Size([1, 10, 12])
torch.Size([1, 2])
tensor(1.)
torch.Size([2])
tensor([1., 0.])
tensor([0., 1.])


In [44]:
#setting loss function and optimizer
loss_functions = [nn.MSELoss()]#, nn.BCELoss(),nn.CrossEntropyLoss()
optimizers = [ torch.optim.SGD]#torch.optim.Adam,
models = [crypto_classfier_ver5()]#crypto_classfier_ver1(), crypto_classfier_ver2(), crypto_classfier_ver3(), crypto_classfier_ver4(), CNNwithDropout(10,2)
history = dict()
lr = 0.0001
epochs = 30
batch_size = 1 #batch size 是一次讀取多少筆資料 我這邊只能設1 因為我們的資料是一筆一筆的
#training
for model in models:
    for loss_function in loss_functions:
        for optimizer in optimizers:
            print("model: ", model.name)
            print("loss_function: ", loss_function)
            print("optimizer: ", optimizer)
            print("lr: ", lr)
            print("epochs: ", epochs)
            print("batch_size: ", batch_size)
            device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
            model.to(device)
            print("device: ", device)
            optimizer = optimizer(model.parameters(), lr=lr)
            train_loss = []
            test_loss = []
            test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)
            train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
            for epoch in range(epochs):
                for i, (x, y) in enumerate(train_loader):
                    model.train()
                    x = x[0]
                    y = y[0]
                    x = x.to(device)
                    y = y.to(device)
                    optimizer.zero_grad()
                    out = model(x)
                    loss = loss_function(out, y)
                    loss.backward()
                    optimizer.step()
                    train_loss.append(loss.item())
                    if i % 1000 == 0:
                        print("Train epoch: ", epoch, "i: ", i, "loss: ", loss.item(), "out: ", out, "y: ", y)
                for i, (x, y) in enumerate(test_loader):
                    model.eval()
                    x = x[0]
                    y = y[0]
                    x = x.to(device)
                    y = y.to(device)
                    out = model(x)
                    loss = loss_function(out, y)
                    test_loss.append([loss.item(), model.parameters()])
                    if i % 1000 == 0:
                        print("epoch: ", epoch, "i: ", i, "loss: ", loss.item())
                if epoch % 10 == 0:
                    print("epoch: ", epoch, "train_loss: ", train_loss[-1], "test_loss: ", test_loss[-1])
            history[model.name + str(loss_function) + str(optimizer)] = [train_loss[-1], test_loss[-1], model.parameters()]



model:  CCV-5
loss_function:  MSELoss()
optimizer:  <class 'torch.optim.sgd.SGD'>
lr:  0.0001
epochs:  30
batch_size:  1
device:  cpu
Train epoch:  0 i:  0 loss:  0.1862603724002838 out:  tensor([0.4316, 0.5684], grad_fn=<SoftmaxBackward0>) y:  tensor([0., 1.])


KeyboardInterrupt: 